In [6]:
import pandas as pd
dataFile='../../../data/bookcrossing/bxbookrating.csv'
data=pd.read_csv(dataFile,sep=",",header=0,names=["user","isbn","rating"])

In [7]:
data.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [8]:
bookFile='../../../data/bookcrossing/bxbook.csv'
books=pd.read_csv(bookFile,sep=",",header=0,error_bad_lines=False, usecols=[0,1,2],index_col=0,names=['isbn',"title","author"])

In [9]:
books.head()

,title,author
isbn,,
195153448,Classical Mythology,Mark P. O. Morford
2005018,Clara Callan,Richard Bruce Wright
60973129,Decision in Normandy,Carlo D'Este
374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
393045218,The Mummies of Urumchi,E. J. W. Barber


In [10]:
def bookMeta(isbn):
    title = books.at[isbn,"title"]
    author = books.at[isbn,"author"]
    return title, author
bookMeta("0671027360")

KeyError: '0671027360'

In [ ]:
data = data[data["isbn"].isin(books.index)]

In [ ]:
def faveBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

In [ ]:
faveBooks(204622,5)

In [ ]:
data.shape

In [ ]:
usersPerISBN = data.isbn.value_counts()
usersPerISBN.head(10)

In [ ]:
usersPerISBN.shape

In [ ]:
ISBNsPerUser = data.user.value_counts()

In [ ]:
ISBNsPerUser.shape

In [ ]:
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)]

In [ ]:
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)]

In [ ]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['user'], columns=['isbn'])

In [ ]:
userItemRatingMatrix.head()

In [ ]:
userItemRatingMatrix.shape

In [ ]:
user1 = 204622
user2 = 255489

In [ ]:
user1Ratings = userItemRatingMatrix.transpose()[user1]
user1Ratings.head()

In [ ]:
user2Ratings = userItemRatingMatrix.transpose()[user2]

In [ ]:
from scipy.spatial.distance import hamming 
hamming(user1Ratings,user2Ratings)

In [ ]:
import numpy as np
def distance(user1,user2):
        try:
            user1Ratings = userItemRatingMatrix.transpose()[user1]
            user2Ratings = userItemRatingMatrix.transpose()[user2]
            distance = hamming(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance 

In [ ]:
distance(204622,10118)

In [ ]:
user = 204622
allUsers = pd.DataFrame(userItemRatingMatrix.index)
allUsers = allUsers[allUsers.user!=user]
allUsers.head()

In [ ]:
allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))

In [ ]:
allUsers.head()

In [ ]:
K = 10
KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user"][:K]

In [ ]:
KnearestUsers

In [ ]:
def nearestNeighbors(user,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.user!=user]
    allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user"][:K]
    return KnearestUsers

In [ ]:
KnearestUsers = nearestNeighbors(user)

In [ ]:
KnearestUsers

In [ ]:
NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
NNRatings

In [ ]:
avgRating = NNRatings.apply(np.nanmean).dropna()
avgRating.head()

In [ ]:
booksAlreadyRead = userItemRatingMatrix.transpose()[user].dropna().index
booksAlreadyRead

In [ ]:
avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]

In [ ]:
N=3
topNISBNs = avgRating.sort_values(ascending=False).index[:N]

In [ ]:
pd.Series(topNISBNs).apply(bookMeta)

In [ ]:
def topN(user,N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    booksAlreadyRead = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]
    topNISBNs = avgRating.sort_values(ascending=False).index[:N]
    return pd.Series(topNISBNs).apply(bookMeta)

In [ ]:
faveBooks(204813,10)

In [ ]:
topN(204813,10)